# 🚀 SPY Option Trader - Cloud Optimizer (GPU)

Run the **Quantor-MTFuzz** optimization suite on Google Colab leveraging T4/A100 GPUs.

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/trextrader/spy-iron-condor-trading/blob/main/colab/spyt_cloud_optimizer.ipynb)

### 📋 Instructions
1.  **Data Setup**: You MUST have your options data (`data/`) uploaded to Google Drive at `My Drive/SPYOptionTrader/data`.
2.  **Runtime**: Select **Runtime > Change runtime type > T4 GPU**.
3.  **Run All**: Execute cells below.

In [ ]:
# 1. Environment Setup
# Clone code from GitHub (latest version) & Mount Drive for Data
import os
import shutil
from google.colab import drive

# A. Clone Code
REPO_URL = "https://github.com/trextrader/spy-iron-condor-trading.git"
WORKSPACE_PATH = "/content/spy-iron-condor-trading"

if os.path.exists(WORKSPACE_PATH):
    shutil.rmtree(WORKSPACE_PATH)

print("⏳ Cloning repository...")
!git clone {REPO_URL}
os.chdir(WORKSPACE_PATH)
print(f"✅ Code loaded at {os.getcwd()}")

# B. Mount Data
drive.mount('/content/drive')
DRIVE_DATA_PATH = '/content/drive/MyDrive/SPYOptionTrader/data'
LOCAL_DATA_PATH = f'{WORKSPACE_PATH}/data'

if os.path.exists(DRIVE_DATA_PATH):
    print("⏳ Linking data from Drive (Zero-Copy mode)...")
    if os.path.exists(LOCAL_DATA_PATH):
        shutil.rmtree(LOCAL_DATA_PATH)
    !ln -s "{DRIVE_DATA_PATH}" "{LOCAL_DATA_PATH}"
    print("✅ Data linked successfully!")
else:
    print(f"⚠️ Data folder not found at {DRIVE_DATA_PATH}. Please upload your CSVs there.")


In [ ]:
# 2. Install Dependencies

# Base requirements
!pip install -r requirements.txt

# GPU Acceleration (Mamba-SSM)
print("⏳ Installing Mamba-SSM & Torch (GPU)...")
!pip install torch==2.1.0+cu121 --index-url https://download.pytorch.org/whl/cu121
!pip install causal-conv1d>=1.2.0
!pip install mamba-ssm>=1.2.0

print("✅ Dependencies installed.")

In [ ]:
# 3. Run Optimization
# Runs the 5-Phase Optimization Matrix

!python core/main.py --mode backtest --use-optimizer --use-mtf --use-fuzzy --dynamic-sizing --no-plot

In [ ]:
# 4. Save Results
import datetime

timestamp = datetime.datetime.now().strftime("%Y%m%d_%H%M%S")
dest_path = f"/content/drive/MyDrive/SPYOptionTrader/reports_colab_{timestamp}"

print(f"⏳ Saving reports to Drive: {dest_path}...")
shutil.copytree('reports', dest_path)
print("✅ Saved!")

# Display Top Result
import pandas as pd
import glob
list_of_files = glob.glob('reports/top100_*.csv') 
if list_of_files:
    latest_file = max(list_of_files, key=os.path.getctime)
    df = pd.read_csv(latest_file)
    display(df.head(5))